In [1]:
!pip install -U langchain langchain-community langchain-openai sentence-transformers faiss-cpu gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [2]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from google.colab import drive
from google.colab import userdata
from pathlib import Path
from tqdm.notebook import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
transcript_folder = Path("/content/drive/MyDrive/ServiceNow_Audio_Transcripts")
transcript_folder.mkdir(parents=True, exist_ok=True)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

documents: list[Document] = []
for txt_path in tqdm(transcript_folder.glob("*.txt"), desc="Chunking transcripts"):
    raw_text = txt_path.read_text(encoding="utf-8")
    for chunk in splitter.split_text(raw_text):
        documents.append(
            Document(page_content=chunk, metadata={"source": txt_path.name})
        )

# 4. Summary
print(f"✅ Loaded and chunked {len(documents)} document chunks from {transcript_folder}.")

Chunking transcripts: 0it [00:00, ?it/s]

✅ Loaded and chunked 656 document chunks from /content/drive/MyDrive/ServiceNow_Audio_Transcripts.


In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_dir = "/content/drive/MyDrive/faiss_store"
faiss_dir = Path(faiss_dir)
Path(faiss_dir).mkdir(parents=True, exist_ok=True)

faiss_index = FAISS.load_local(
    faiss_dir,
    embedding_model,
    allow_dangerous_deserialization=True
)

# 4. Now you can query it or wrap it in a retriever/chain—no save_local() needed here
retriever = faiss_index.as_retriever(search_kwargs={"k": 5})

/tmp/ipython-input-5-2449386041.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
llm = OpenAI(
    temperature=0,
    openai_api_key=userdata.get("OPENAI_API_KEY")
)

retriever = faiss_index.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [16]:
from typing import Tuple, Union
import gradio as gr
from transformers import pipeline

# Load the speech-to-text model (changed to Whisper small)
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-small")

def answer_question(text_input: str, audio_input: Union[str, dict, None]) -> Tuple[str, str]:
    """
    Runs the RetrievalQA chain and formats the answer along with source snippets.
    Handles both text and audio input.

    Args:
        text_input: The string input from the text box.
        audio_input: The audio input, expected as a string filepath from Gradio.

    Returns:
      - answer text
      - formatted source list (one entry per document)
    """
    try:
        question = ""
        print(f"Debug: text_input type: {type(text_input)}, value: '{text_input}'")
        print(f"Debug: audio_input type: {type(audio_input)}, value: {audio_input}")


        # Process audio input if provided (handle as a string filepath)
        if isinstance(audio_input, str) and audio_input:
             audio_filepath = audio_input
             print(f"Debug: Processing audio from filepath: {audio_filepath}")
             try:
                 transcription_result = transcriber(audio_filepath)
                 print(f"Debug: Transcription result: {transcription_result}")
                 question = transcription_result.get("text", "")
                 if not question:
                     print("Debug: Transcription resulted in empty string.")
                     return "Could not transcribe audio. Please try again.", ""
                 question = question.strip() # Remove leading/trailing whitespace from transcription
                 if not question:
                     print("Debug: Stripped transcription is empty.")
                     return "Transcribed audio was empty. Please try again.", ""
             except Exception as trans_e:
                 print(f"Debug: Error during transcription: {trans_e}")
                 return f"Error during transcription: {trans_e}", ""


        # Use text input if audio was not provided or transcription failed
        if not question and isinstance(text_input, str) and text_input.strip():
            print("Debug: Using text input.")
            question = text_input.strip()
            if not question:
                print("Debug: Stripped text input is empty.")
                return "Please enter a question or record audio.", ""

        # Ensure there is a question to process after handling input types
        if not question:
             print("Debug: Final question is empty after checking both inputs.")
             return "No valid question provided via text or audio.", ""

        print(f"Debug: Final question for QA chain: '{question}'")

        # Run the QA chain with the processed question
        try:
            print("Debug: Calling qa_chain...")
            response = qa_chain(question)
            print(f"Debug: qa_chain response type: {type(response)}")
            print(f"Debug: qa_chain response keys: {response.keys() if isinstance(response, dict) else 'Not a dict'}")
            answer = response.get("result", "") # Use .get() with a default value
            print(f"Debug: Answer extracted: '{answer}')")

            # Build source list with preview snippets
            sources = []
            source_documents = response.get("source_documents", []) # Use .get() with a default empty list
            print(f"Debug: Number of source documents: {len(source_documents)}")
            for doc in source_documents:
                text = doc.page_content.strip()
                preview = text[:400] + "..." if len(text) > 400 else text
                sources.append(f"📄 {doc.metadata.get('source', 'unknown')}\n🔎 {preview}")
            print(f"Debug: Sources formatted. Number of formatted sources: {len(sources)}")

            if not answer:
                 print("Debug: Answer is empty after qa_chain.")
                 # It's possible the QA chain returned successfully but with no relevant answer
                 return "Could not find a relevant answer in the documents.", "\n\n".join(sources)


            return answer, "\n\n".join(sources)

        except Exception as qa_e:
            print(f"Debug: Error during qa_chain execution or response processing: {qa_e}")
            return f"Error processing question with QA chain: {qa_e}", ""


    except Exception as e:
        err = f"An unexpected error occurred in answer_question: {e}"
        print(f"Debug: Unexpected error in answer_question: {e}") # Print error for debugging
        # Return the same error in both slots to display in UI
        return err, err

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [26]:
code = '''import os
from typing import Tuple

import gradio as gr
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Securely load your API key
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise EnvironmentError("OPENAI_API_KEY not set in environment")

# 2. Sample documents (replace with your FAISS index load in production)
documents = [
    Document(
        page_content="ServiceNow uses AI for incident classification.",
        metadata={"source": "intro.txt"}
    )
]

# 3. Build FAISS index (for production, load existing index instead)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_index = FAISS.from_documents(documents, embedding=embedding_model)

# 4. Initialize the OpenAI LLM
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

# 5. Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    return_source_documents=True
)

# 6. Define your Gradio interface function
def answer_question(question: str) -> Tuple[str, str]:
    """
    Returns:
      - The generated answer
      - A string of source filenames + snippet previews
    """
    response = qa_chain(question)
    answer = response["result"]

    sources = []
    for doc in response["source_documents"]:
        text = doc.page_content.strip()
        preview = text[:200] + "..." if len(text) > 200 else text
        sources.append(f"📄 {doc.metadata.get('source', 'unknown')}\n🔎 {preview}")

    return answer, "\n\n".join(sources)

gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a ServiceNow question..."),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Sources")
    ],
    title="ServiceNow QA Bot",
    description="A RAG-powered assistant over ServiceNow transcripts.",
    allow_flagging="never"
).launch()
'''

with open("app.py", "w") as f:
    f.write(code)

In [27]:
requirements = '''gradio
langchain
langchain-openai
langchain-community
langchain-core
sentence-transformers
faiss-cpu
openai
yt-dlp
python-dotenv
'''

with open("requirements.txt", "w") as f:
    f.write(requirements)

In [28]:
from google.colab import files
files.download("app.py")
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import gradio as gr

gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(lines=2, placeholder="Ask a ServiceNow question via text...", label="Text Input"), # Text input
        gr.Audio(sources=["microphone"], type="filepath", label="Ask your question via microphone") # Audio input
    ],
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Sources")
    ],
    title="ServiceNow QA Assistant",
    description="RAG-powered assistant over your ServiceNow YouTube transcripts with text and speech input.", # Updated description
    allow_flagging="never"
).launch(
    share=True,
    debug=True,
    inline=False,
    show_error=True,
    prevent_thread_lock=True
)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1782ca3817557cd320.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Debug: text_input type: <class 'str'>, value: ''
Debug: audio_input type: <class 'str'>, value: /tmp/gradio/b381a7a3f0e4985bd704ada0021ec76025456ee3945332f999c4085d8a9279cd/audio.wav
Debug: Processing audio from filepath: /tmp/gradio/b381a7a3f0e4985bd704ada0021ec76025456ee3945332f999c4085d8a9279cd/audio.wav


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Debug: Transcription result: {'text': ' What is service now?'}
Debug: Final question for QA chain: 'What is service now?'
Debug: Calling qa_chain...
Debug: qa_chain response type: <class 'dict'>
Debug: qa_chain response keys: dict_keys(['query', 'result', 'source_documents'])
Debug: Answer extracted: ' ServiceNow is a cloud-based platform that provides a variety of IT and business management services, including IT service management, human resources, customer service, and security operations. It is designed to help organizations streamline their processes, improve efficiency, and deliver better services to their customers.')
Debug: Number of source documents: 5
Debug: Sources formatted. Number of formatted sources: 5
Debug: text_input type: <class 'str'>, value: 'What is ServiceNow?'
Debug: audio_input type: <class 'NoneType'>, value: None
Debug: Using text input.
Debug: Final question for QA chain: 'What is ServiceNow?'
Debug: Calling qa_chain...
Debug: qa_chain response type: <class 

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Debug: Transcription result: {'text': ' What is CMDB in service now?'}
Debug: Final question for QA chain: 'What is CMDB in service now?'
Debug: Calling qa_chain...
Debug: qa_chain response type: <class 'dict'>
Debug: qa_chain response keys: dict_keys(['query', 'result', 'source_documents'])
Debug: Answer extracted: ' CMDB stands for Configuration Management Database and it is a feature in ServiceNow that serves as a system of record for enterprise data. It helps with tracking and managing IT assets, services, and configurations.')
Debug: Number of source documents: 5
Debug: Sources formatted. Number of formatted sources: 5
